<a href="https://colab.research.google.com/github/KVVisDon/Data-Mining-Project/blob/main/code/Team_Microsoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Team Microsoft

## 1: Import packages

In [42]:
import pandas as pd 
import seaborn as sns
import numpy as np
import re
import string
import sklearn
import csv,json

import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%matplotlib inline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stpw

#sns.set_style("white")
#%load_ext autoreload
#%autoreload 2
#from sklearn import datasets


#%load_ext autoreload
#%autoreload 2

In [39]:
df = pd.read_csv("https://raw.githubusercontent.com/KVVisDon/Data-Mining-Project/main/data/training_data.csv")
df

df_test = pd.read_csv("https://raw.githubusercontent.com/KVVisDon/Data-Mining-Project/main/data/test_data.csv")

## 1.2: Base rate

In [33]:
br = df.target.value_counts().max()/df.target.value_counts().sum()
number_per_class = df.target.value_counts()
print(number_per_class)
print("Base rate is",str(round(br,2)))

0    3701
1    2770
Name: target, dtype: int64
Base rate is 0.57


# 2: Cleaning
###Basic : first we defined a tokenzation function, and removed duplicates based on subset ['text','keyword']

In [34]:
#Tokenization function : lower-case, lemmatizing, remove "stop words", remove "punctuations"
sp = spacy.load('en_core_web_sm')
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spa_tok(message):
  mytokens = sp(message)

# Lemmatize each token and convert each token into lowercase
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

# Remove stop words and punctuation
  mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

# Remove hashtags, @

  mytokens = [ word.replace('@','').replace('#', '') for word in mytokens ]

  mytokens = [ word.replace('rt @', '').replace('rt', '') for word in mytokens ]
     
  mytokens = [ word for word in mytokens if word not in [""] ]

  mytokens = [ word for word in mytokens if word not in [""] ]

  return mytokens

#remove duplicates (check the subset parameter)
df = df.drop_duplicates(subset=['text','keyword'], keep='first') 

#Tokenization function test
print(spa_tok("RT @meljomur: you know there's something seriously puts a climate disaster that wll hardly blabla!!! #fdf"))

#To improve performance : drop unused columns ('id' for now)
df = df.drop(columns=['id'])
df

['meljomur', 'know', 'seriously', 'climate', 'disaster', 'wll', 'hardly', 'blabla', 'fdf']


,keyword,location,text,target
0,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1
...,...,...,...,...
6466,earthquake,ARGENTINA,#Earthquake #Sismo M 1.9 - 15km E of Anchorage...,1
6467,derail,NaN,@EmiiliexIrwin Totally agree.She is 23 and kno...,0
6468,trapped,NaN,Hollywood Movie About Trapped Miners Released ...,1
6469,weapons,Beirut/Toronto,Friendly reminder that the only country to eve...,1


# 3: Train a model
###Basic : 
- split train/test (80/20)
- define TF-IDF vectorizer
- define classifier
- fit model on training set
- first evaluation

In [37]:
X = df['text']
y = df['target']

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=59)

X_train

# Define vectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), tokenizer=spa_tok)

# Define classifier
classifier = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=72)

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf),
                ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)


nFeat = 0
for i in tfidf.get_feature_names():
  nFeat += 1
print("number of features created by the TF-IDF vector :",str(nFeat))

y_pred = pipe.predict(X_test)

#first evaluation
print(f"ACCURACY SCORE test:\n{accuracy_score(y_test, y_pred):.3f}")

print(f"ACCURACY SCORE train:\n{accuracy_score(y_train, pipe.predict(X_train)):.3f}")

print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_pred)}")

number of features created by the TF-IDF vector : 2229
ACCURACY SCORE train:
0.855
CONFUSION MATRIX:
[[633  83]
 [183 382]]


# 4: First submission
###Basic : 
- mount the drive in order to save the csv with submission values
- split train/test (80/20)
- define TF-IDF vectorizer
- define classifier
- fit model on training set

In [48]:
#Mount a google drive
from google.colab import drive

drive.mount('drive')


#To improve performance : drop unused columns ('id' for now)
test_data = df_test['text']


# Get & print the number of features generated by the TF-IDF vector
nFeat = 0
for i in tfidf.get_feature_names():
  nFeat += 1
print("number of features created by the TF-IDF vector :",str(nFeat))

test_pred = pipe.predict(test_data)
print(type(test_pred))
print(test_pred)

#first evaluation
print(f"ACCURACY SCORE test:\n{accuracy_score(y_test, y_pred):.3f}")

print(f"ACCURACY SCORE train:\n{accuracy_score(y_train, pipe.predict(X_train)):.3f}")

print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_pred)}")

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
number of features created by the TF-IDF vector : 2229
<class 'numpy.ndarray'>
[0 0 1 ... 1 0 1]
ACCURACY SCORE test:
0.792
ACCURACY SCORE train:
0.855
CONFUSION MATRIX:
[[633  83]
 [183 382]]


In [49]:
#Handle the submission file
submission_file = pd.DataFrame(test_pred,columns=['target'])

submission_file.to_csv('submission_one.csv', index=False)

!cp submission_one.csv "drive/My Drive/Team_Microsoft"